In [1]:
from tabtransformertf.models.tabtransformer import TabTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset, build_categorical_prep

2023-01-14 09:46:20.155204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 09:46:20.294455: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-14 09:46:20.299121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:
2023-01-14 09:46:20.299134: I tensorflow/compiler/xla/s

In [2]:
import pandas as pd

In [3]:
X_train= pd.read_csv('/media/Z/NDT/Transformer_Tabular/dataset/train.csv')
X_val=pd.read_csv('/media/Z/NDT/Transformer_Tabular/dataset/val.csv')
test_data=pd.read_csv('/media/Z/NDT/Transformer_Tabular/dataset/test.csv')

In [4]:
CATEGORICAL_FEATURES=['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'gender', 'native_country']
NUMERIC_FEATURES=['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week']
LABEL = 'income_bracket'

In [5]:
# Category preprocessing layers
category_prep_layers = build_categorical_prep(X_train, CATEGORICAL_FEATURES)

  0%|          | 0/8 [00:00<?, ?it/s]2023-01-14 09:46:23.507208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:
2023-01-14 09:46:23.507869: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:
2023-01-14 09:46:23.508352: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:
2023-01-14 09:46:23.508796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc

In [6]:
FEATURES = CATEGORICAL_FEATURES + NUMERIC_FEATURES

In [7]:
# To TF Dataset
train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL)
val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
test_dataset = df_to_dataset(test_data[FEATURES + [LABEL]], shuffle=False) # No target, no shuffle

/home/administrator/.virtualenvs/ndt98/lib/python3.8/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/home/administrator/.virtualenvs/ndt98/lib/python3.8/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]


In [8]:
tabtransformer = TabTransformer(
    numerical_features = NUMERIC_FEATURES,
    categorical_features = CATEGORICAL_FEATURES,
    categorical_lookup=category_prep_layers,
    embedding_dim=32,
    out_dim=1,
    out_activation='sigmoid',
    depth=4,
    heads=8,
    attn_dropout=0.2,
    ff_dropout=0.2,
    mlp_hidden_factors=[2, 4],
    use_column_embedding=True,
)

In [9]:
import tensorflow as tf
import tensorflow_addons as tfa

In [10]:
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.0001
NUM_EPOCHS = 1000

optimizer = tfa.optimizers.AdamW(
        learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

tabtransformer.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics= [tf.keras.metrics.AUC(name="PR AUC", curve='PR')],
)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
early = EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)
callback_list = [early]

history = tabtransformer.fit(
    train_dataset, 
    epochs=NUM_EPOCHS, 
    validation_data=val_dataset,
    callbacks=callback_list
)

Epoch 1/1000
51/51 [==============================] - 12s 134ms/step - loss: 0.7687 - PR AUC: 0.4415 - val_loss: 0.6561 - val_PR AUC: 0.6140
Epoch 2/1000
51/51 [==============================] - 6s 122ms/step - loss: 0.6022 - PR AUC: 0.5393 - val_loss: 0.4649 - val_PR AUC: 0.6440
Epoch 3/1000
51/51 [==============================] - 6s 124ms/step - loss: 0.5214 - PR AUC: 0.5826 - val_loss: 0.4262 - val_PR AUC: 0.6588
Epoch 4/1000
51/51 [==============================] - 6s 120ms/step - loss: 0.4800 - PR AUC: 0.6048 - val_loss: 0.3870 - val_PR AUC: 0.6669
Epoch 5/1000
51/51 [==============================] - 6s 120ms/step - loss: 0.4438 - PR AUC: 0.6278 - val_loss: 0.3788 - val_PR AUC: 0.6734
Epoch 6/1000
51/51 [==============================] - 6s 120ms/step - loss: 0.4265 - PR AUC: 0.6374 - val_loss: 0.3752 - val_PR AUC: 0.6786
Epoch 7/1000
51/51 [==============================] - 6s 124ms/step - loss: 0.4182 - PR AUC: 0.6479 - val_loss: 0.3695 - val_PR AUC: 0.6835
Epoch 8/1000
51/51 

In [13]:
test_preds = tabtransformer.predict(test_dataset)

32/32 [==============================] - 2s 44ms/step


In [14]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

In [16]:
print("Test ROC AUC:", np.round(roc_auc_score(test_data[LABEL], test_preds.ravel()), 4))
print("Test PR AUC:", np.round(average_precision_score(test_data[LABEL], test_preds.ravel()), 4))
print("Test Accuracy:", np.round(accuracy_score(test_data[LABEL], test_preds.ravel() > 0.5), 4))

Test ROC AUC: 0.8956
Test PR AUC: 0.7352
Test Accuracy: 0.8458
